In [1]:
import os
import sys

In [2]:
%pwd

'c:\\Users\\chenk\\OneDrive\\Desktop\\end_to_end classifier\\chicken_disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\chenk\\OneDrive\\Desktop\\end_to_end classifier\\chicken_disease_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from Chicken_disease_classification.constants import *
from Chicken_disease_classification.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        return data_ingestion_config
    

In [8]:
import os
import urllib.request as request
import zipfile
import py7zr
from Chicken_disease_classification import logger
from Chicken_disease_classification.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        # For .7z extraction
        if self.config.local_data_file.endswith(".7z"):
            with py7zr.SevenZipFile(self.config.local_data_file, mode="r") as archive:
                archive.extractall(path=unzip_path)
            print(f"✅ Extracted .7z to {unzip_path}")

        # Optional: if you also want to handle .zip in future
        elif self.config.local_data_file.endswith(".zip"):
            import zipfile
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            print(f"✅ Extracted .zip to {unzip_path}")

        else:
            raise ValueError("Unsupported file type for extraction")

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_file()
except Exception as e:
    raise e

[2025-09-14 18:24:25,935: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-14 18:24:25,940: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-14 18:24:25,943: INFO: common: created directory at: artifacts]
[2025-09-14 18:24:25,945: INFO: common: created directory at: artifacts/data_ingestion]
